In [ ]:
#test code
# Generate images from noise, using the generator network.
f, a = plt.subplots(4, 10, figsize=(10, 4))
for i in range(10):
    # Noise input.
    z = np.random.uniform(-1., 1., size=[4, noise_dim])
    g = sess.run(gen_sample, feed_dict={noise_input: z})
    for j in range(4):
        # Generate image from noise. Extend to 3 channels for matplot figure.
        img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2),
                         newshape=(28, 28, 3))
        a[j][i].imshow(img)

In [ ]:
#test code
if step == 0 or (step + 1) % 10 == 0:
    generated_samples = model.sample_generator(noise_test, batch_size=visualization_num)
    savepath = output_dir + '/output_' + 'EP' + str(epoch).zfill(3) + "_Batch" + str(step).zfill(6) + '.jpg'
    save_visualization(generated_samples, (14, 14), save_path=savepath)

In [ ]:
def save_visualization(X, nh_nw, save_path='./vis/sample.jpg'):
    nh, nw = nh_nw
    h, w = X.shape[1], X.shape[2]
    img = np.zeros((h * nh, w * nw, 3))

    for n, x in enumerate(X):
        j = int(n / nw)
        i = int(n % nw)
        img[j * h:j * h + h, i * w:i * w + w, :] = x

    cv2.imwrite(save_path, img)

In [ ]:
def sample_generator(self, noise_z, batch_size=1):
        noise_z = np.array(noise_z).reshape([batch_size, self.dim_z])

        Z = tf.placeholder(tf.float32, [batch_size, self.dim_z])
        h1 = tf.reshape(Z, [batch_size, 1, 1, self.dim_z])
        h1 = tf.nn.conv2d_transpose(h1, self.G_W1, output_shape=[batch_size, 4, 4, self.dim_W1],
                                    strides=[1, 4, 4, 1])
        h1 = tf.nn.relu(self.G_bn1(h1))

        output_shape_l2 = [batch_size, 8, 8, self.dim_W2]
        h2 = tf.nn.conv2d_transpose(h1, self.G_W2, output_shape=output_shape_l2, strides=[1, 2, 2, 1])
        h2 = tf.nn.relu(self.G_bn2(h2))

        output_shape_l3 = [batch_size, 16, 16, self.dim_W3]
        h3 = tf.nn.conv2d_transpose(h2, self.G_W3, output_shape=output_shape_l3, strides=[1, 2, 2, 1])
        h3 = tf.nn.relu(self.G_bn3(h3))

        output_shape_l4 = [batch_size, 32, 32, self.dim_W4]
        h4 = tf.nn.conv2d_transpose(h3, self.G_W4, output_shape=output_shape_l4, strides=[1, 2, 2, 1])
        h4 = tf.nn.relu(self.G_bn4(h4))

        output_shape_l5 = [batch_size, 64, 64, self.dim_W5]
        h5 = tf.nn.conv2d_transpose(h4, self.G_W5, output_shape=output_shape_l5, strides=[1, 2, 2, 1])

        x = tf.nn.tanh(h5)

        generated_samples = self.sess.run(x, feed_dict={Z: noise_z})
        generated_samples = (generated_samples + 1.) / 2.
        return generated_samples
